In [3]:
# !pip install import_ipynb
import import_ipynb
import Classification_Function 
from imp import reload
reload(Classification_Function)
from Classification_Function import *
import tensorflow as tf

importing Jupyter notebook from Classification_Function.ipynb
importing Jupyter notebook from Classification_Function.ipynb


In [4]:
df=pd.read_excel("농축 학습데이터 계통 및 형태소 추출 ver1.2.xlsx")

df["Label"]=df["a_prices_prdlst_nm"]+" "+df["정답여부"]

df_1=df[df["구분"]=='계통']

df_2=df[df["구분"]=='형태소']

df_1=df_1[['구분',"goods_nm","Label"]]

df_2=df_2[['구분',"goods_nm","Label"]]

In [9]:
df_2

,구분,goods_nm,Label
0,형태소,#10/6 49_900원# [푸드조아] 20년 햅쌀 황금미가 20kg,쌀 Y
1,형태소,#6/23 단하루 39_900원# [홍천철원] 19년산 햇빛담은쌀 20kg,쌀 Y
2,형태소,#6/26~ 중소기업쿠폰+현대카드20%쿠폰 최종가 34_900원# [명가미곡]지리산...,쌀 Y
3,형태소,('19년 햅쌀) 굿뜨래 평안쌀(10KG),쌀 Y
4,형태소,(17년 햅쌀) 경기도 대왕님표 여주쌀 4kg,쌀 Y
...,...,...,...
145146,형태소,펠리칸 LED 방폭라이트 1965 21Lm 140x19mm 2xAAA (1EA),생화 N
145147,형태소,펠리칸 LED 방폭라이트 2010 109Lm 178x45mm 3xC (1EA),생화 N
145148,형태소,펠리칸 LED 방폭라이트 2410 126Lm 178x45mm 4xAA (1EA),생화 N
145149,형태소,펠리칸 LED라이트 4000 251Lm 12 D형x8EA (1EA),생화 N


In [10]:
df_1.to_csv("Data_계통.csv", sep=",", encoding="utf-8")
df_2.to_csv("Data_형태소.csv", sep=",", encoding="utf-8")

In [15]:
Label_1=df_1["Label"]

# 라벨 중복제거
Label_list=Label_1.unique()

# index = 번호 , 
Label_df_1=pd.DataFrame(index=range(0,len(Label_list)), columns=["Index","Label"])

In [16]:
Label_2=df_2["Label"]

Label_list2=Label_2.unique()

Label_df_2=pd.DataFrame(index=range(0,len(Label_list2)), columns=["Index","Label"])

In [17]:
for i in tqdm(range(len(Label_list))):
    Label_df_1["Label"][i]=Label_list[i]
    Label_df_1["Index"][i]=i

100%|██████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 6642.24it/s]


In [18]:
for i in tqdm(range(len(Label_list2))):
    Label_df_2["Label"][i]=Label_list2[i]
    Label_df_2["Index"][i]=i

100%|██████████████████████████████████████████████████████████████████████████████| 146/146 [00:00<00:00, 8610.60it/s]


In [15]:
Label_df_1.to_csv("Data_계통_Label.csv",sep=",", encoding="utf-8-sig" )
Label_df_2.to_csv("Data_형태소_Label.csv",sep=",", encoding="utf-8-sig" )

In [26]:
# sample 무작위 심플 추출 / frac = 0.1 ~ 1 전체 개수의 비율 만큼 샘플을 반환하려 할 경우 사용 됨 
# frac = 1 / 무작위 재배치
df_1=df_1.sample(frac=1).reset_index(drop=True)

# (Classification_Function) 쓸모없는 기호 제거 및 중복된 상품명 제거
df_1=text_preprocessing(df_1)

# na값 제거
df_1=df_1.dropna()

# stopwods 제거
stop = pd.read_table("stopwords_korean.txt", sep="\t")

# stopwords(배제어) list 생성
stopwords=stop["이"].tolist()

# (Classification_Function) 형태소 분석
X_train=tokenizer(df_1,stopwords)

# 분석을 위해 list 형태를 array 형태로 변환 
y_train=np.array(df_1["Label"])


X_dict={}
XX_train,X_dict=vocab_indexing(X_train)     
XX_train

# 상품명이 적혀있지 않은 데이터 인덱스를 가져온다.
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

NameError: name 'vocab_indexing' is not defined

In [49]:
# 결측행 제거
XX_train = np.delete(XX_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

yy_train=[]
yy_train,den=Labeling_var(y_train,'Label')

# pad_sequences 길이가 같지 않고 적거나 많을 때 일정한 길이로 맞춰주기 위해 사용한다.
# maxlen = 32 : 시퀀스의 최대길이를 32로 지정하고 초과한 경우 각 시퀀스의 앞쪽에서 자른다.
# why 32 : 
XX_train = pad_sequences(XX_train, maxlen = 32)

import time # 시간 관련 패키지
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D
from keras import optimizers

# 딥러닝 구조를 결정한다(모델을 설정하고 실행하는 부분)
# sequential: 층을 쌓을 수 있게 한다.
# Embedding: 워드 임베딩이란 텍스트 내의 단어들을 밀집 벡터(dense vector)로 만드는 것.
# Embedding(불러온 단어의 총 개수(단어사전의 크기), 상품명 당 단어 수)
# LSTM(상품명 당 단어 수, 기타 옵션)
model = Sequential()
model.add(Embedding(50000, 128,)) 
model.add(LSTM(128))
model.add(Dense(den, activation='softmax'))

# https://3months.tistory.com/424
# EarlyStopping: 성능이 더이상 증가하지 않을 때 학습을 중지시키게 한다.
# monitor: 손실 혹은 정확도 중 어떤 것을 기준으로 할지 설정하는 역할
# val_loss: validation_loss 손실
# mode: min, mas ex) 손실을 최소화 시키는 방향으로 training이 진행될 때
# verbose=1: 언제 keras에서 training을 멈추었는지 화면에 출력
# patience: 성능이 증가하지 않는 epoch을 몇 번이나 허용할 것인가 정의 (주관적인 기준)
# modelCheckpoint: 모델 저장하기 위한 함수
# save_best_only: 가장 좋은 모델만 저장. (h5는 모델 저장할 때 붙는 확장자 같은 역할)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
checkpoint_path='train_model_adam_1109_농축수산.h5'
mc = ModelCheckpoint(chekpoint_path, monitor='val_acc', mode='max', verbose=1, save_best_only=True)


# model.compile: 앞서 지정한 모델이 효과적으로 구현될 수 있게 여러가지 환경을 설정해 주면서 컴파일하는 부분.
# optimizer: 최적화 방법
# optimizer='adam': adam은 현재 가장 많이 사용되는 고급 경사 하강법
# loss: 오챠 함수
# loss='sparse_categorical_crossentropy': 3개 이상의 범주형 교차 엔트로피(일반적인 분류)
# metrics: 모델이 컴파일될 때 모델 수행 결과를 나타내게끔 설정하는 부분
# metrics=['acc']: 정확도 출력
start = time.time()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
# cpu 동작
with tf.device('/cpu:0'):
    # fit: 모델 수행
    # epochs: 모든 샘플에 대해 한 번 실행되는 것
    # callbacks: epochs가 끝날 때마다의 중간 저장기능
    # batch_size: 샘플을 한 번에 몇 개씩 처리할 지를 정해준다. ex) batch_size=32:전체 샘플을 32개씩 끊어서 집어넣어라 
    # validation_split: XX_train과 yy_train에서 검증데이터로 사용하기 위한 분리 비율
    history = model.fit(XX_train, yy_train, epochs=100, callbacks=[es, mc], batch_size=32, validation_split=0.1)

# time.time() - start : 현재시각 - 시작시간 = 실행 시간
print("time :", time.time() - start)